In [1]:
from math import sqrt
from numpy import concatenate
import matplotlib as mpl
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from keras.models import Sequential
from tensorflow.keras.models import load_model
from keras.layers import LSTM, Dense, Masking
from tensorflow.keras import callbacks as cb
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
from hydro_helpers.scoring import render_score
from hydro_helpers.reporting import cv_report
from hydro_helpers._utils import _nse as NSE
from hydro_helpers._utils import _rmse as RMSE
mpl.rcParams['figure.figsize'] = (20, 10)
mpl.rcParams['axes.grid'] = False
tf.random.set_seed(13)

Using TensorFlow backend.


In [4]:
river = 'lena'
df = pd.read_csv('data/lena/train_df.txt', index_col='date')
print(df.shape)
df.index = pd.DatetimeIndex(df.index)
# print(df.mean(axis=0))
df.head()

(12207, 211)


,stage_max_3019,stage_max_3021,stage_max_3024,stage_max_3027,stage_max_3028,stage_max_3029,stage_max_3030,stage_max_3031,stage_max_3032,stage_max_3035,...,prec_30372,prec_30385,prec_30393,prec_30433,prec_30471,prec_30493,prec_31004,prec_31026,prec_31102,prec_31137
date,,,,,,,,,,,,,,,,,,,,,
1985-01-01,-23.0,103.0,106.0,25.0,258.0,51,126.0,152.0,143,59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.2,0.0
1985-01-02,-23.0,103.0,106.0,23.0,258.0,49,122.0,150.0,141,58,...,0.0,0.0,0.0,0.9,0.0,0.0,0.5,0.7,0.0,0.7
1985-01-03,-23.0,102.0,106.0,21.0,255.0,47,120.0,147.0,140,57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.6,1.6,0.0,0.5
1985-01-04,-24.0,102.0,106.0,19.0,252.0,45,119.0,146.0,139,55,...,0.0,0.0,0.6,0.3,0.0,0.2,3.0,0.2,0.3,0.3
1985-01-05,-24.0,102.0,106.0,17.0,251.0,43,119.0,145.0,135,53,...,0.0,0.0,0.4,0.0,0.0,0.3,0.6,0.0,0.2,1.8


In [5]:
# run once
df = df.dropna(axis=0, how='all')
df = df.dropna(axis=1, how='all')
# print(df.isna().sum())
print(df.shape)
del df['stage_max_3048']
df = df.apply(lambda x: x.fillna(x.mean()),axis=0)
# print(df.isna().sum())
df = df.apply(lambda x: x.fillna(0),axis=0)
print(df.isna().sum())

(12207, 192)
stage_max_3019        0
stage_max_3021        0
stage_max_3024        0
stage_max_3027        0
stage_max_3028        0
stage_max_3029        0
stage_max_3030        0
stage_max_3031        0
stage_max_3032        0
stage_max_3035        0
stage_max_3036        0
stage_max_3037        0
stage_max_3038        0
stage_max_3041        0
stage_max_3042        0
stage_max_3045        0
stage_max_3047        0
stage_max_3050        0
stage_max_3087        0
stage_max_3106        0
stage_max_3169        0
stage_max_3180        0
stage_max_3229        0
stage_max_3230        0
stage_max_3554        0
stage_max_3555        0
ice_thickness_3019    0
ice_thickness_3021    0
ice_thickness_3024    0
ice_thickness_3027    0
                     ..
prec_24763            0
prec_24817            0
prec_24923            0
prec_24933            0
prec_24944            0
prec_24951            0
prec_24959            0
prec_24966            0
prec_24967            0
prec_30028            0
pre

In [ ]:
# df.plot(subplots=True, figsize=[20,20])
# plt.show()
corr = round(df.corr(), 2)
# целевые гидропосты
# rownames = [k for k in corr.columns[corr.columns.str.contains("3019|3027|3028|3029|3030|3035|3041|3045|3050|3230|3036")].values]
rownames = [f'stage_max_{i}' for i in [3019,3027,3028,3029,3030,3035,3041,3045,3050,3230,3036]]
# позиция целевых гидропостов
# positions = [i for i, x in enumerate(corr.columns.str.contains("3019|3027|3028|3029|3030|3035|3041|3045|3050|3230|3036]")) if x]
# print(positions)
corr.to_excel('results/lena_kirensk_corr.xlsx')
corr.loc[rownames, :].transpose().style.background_gradient(cmap='coolwarm')
# corr.style.background_gradient(cmap='coolwarm')

In [8]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])
        
    return np.array(data), np.array(labels)


def make_dataset(df, frac, target_label):
    test_size = len(df) - int(len(df) * frac)
    print('Train: ', test_size)
    print('Test: ', len(df) - test_size)
    dataset = df.values
    data_mean = np.nanmean(dataset[:test_size], axis=0)
    data_std = np.nanstd(dataset[:test_size], axis=0)
    print(dataset[:test_size].shape)
#     print(np.round(data_mean, 3), np.round(data_std, 2))
    dataset = (dataset-data_mean)/data_std
#     print(dataset[:,df.columns.get_loc(target_label)])
    # shaping training and test dataset to chunks
    x_train_multi, y_train_multi = multivariate_data(dataset[:, :], dataset[:, df.columns.get_loc(target_label)], len(dataset) - test_size,
                                                     None, past_history,
                                                     future_target, STEP)
    x_val_multi, y_val_multi = multivariate_data(dataset[:, :], dataset[:, df.columns.get_loc(target_label)],
                                                 0, int(len(df) * frac), past_history,
                                                 future_target, STEP)
    print('\n Input training data shape:')
    print(x_train_multi.shape[-2:], y_train_multi.shape[-2:])
    print('\n Input test data shape:')
    print(x_val_multi.shape[-2:], y_val_multi.shape[-2:])
#     print('\n Single step history:')
#     print(x_train_multi[0])
#     print('\n Single step target:')
#     print(y_train_multi[0])
    return x_train_multi, y_train_multi, x_val_multi, y_val_multi, data_mean, data_std


def model_build(df, target_label):
    print(target_label)
    x_train_multi, y_train_multi, x_val_multi, y_val_multi, data_mean, data_std = make_dataset(df, 0.2)
    
    # create train and test data from chunks
    train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
    val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))

    # training and test data shuffling
    train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

    multi_step_model = tf.keras.models.Sequential()
    multi_step_model.add(tf.keras.layers.LSTM(50,
                                              return_sequences=False,
                                              input_shape=x_train_multi.shape[-2:]))
    multi_step_model.add(tf.keras.layers.Dense(7))

    multi_step_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, 
                                                                beta_2=0.999, epsilon=1e-07, amsgrad=False,
                                                                name='Adam'), 
                             loss='mse')

    model_callbacks = [cb.EarlyStopping(patience=10), 
                              cb.ModelCheckpoint(filepath=f"output/"+ river + "_" + target_label + "_LSTM.h5", save_best_only=True)]
    multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=10,
                                          callbacks=model_callbacks)
    loss = multi_step_history.history['loss']
    val_loss = multi_step_history.history['val_loss']
    epochs = range(len(loss)) 
    plt.figure(figsize=[10,10])
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.legend()
    plt.show()

In [6]:
# define train/test split
target_labels = [f'stage_max_{i}' for i in [3019,3027,3028,3029,3030,3035,3041,3045,3050,3230,3036]]
print(target_labels)
# lookback parameters
past_history = 60 # how many instances a model sees for training, in compliance with other models' input
future_target = 7 # how many instances form a prediction step
STEP = 1 # step size - 1 for daily data
BATCH_SIZE = 30 
BUFFER_SIZE = 1000
EVALUATION_INTERVAL = len(df.values)/BATCH_SIZE # what data model sees for training
EPOCHS = 100 # how many times a model sees the data

['stage_max_3019', 'stage_max_3027', 'stage_max_3028', 'stage_max_3029', 'stage_max_3030', 'stage_max_3035', 'stage_max_3041', 'stage_max_3045', 'stage_max_3050', 'stage_max_3230', 'stage_max_3036']


In [ ]:
# run model training and saving
for l in target_labels:
    model_build(df, l)

In [120]:
# new data
new_df = pd.read_csv('data/lena/df_input_cp3.txt', index_col='date')
new_df.index = pd.DatetimeIndex(new_df.index)
new_df = new_df.dropna(axis=1, how='all')
del new_df['stage_max_3048']
print(new_df.shape)
new_df = new_df.apply(lambda x: x.fillna(x.mean()),axis=0)
new_df = new_df.apply(lambda x: x.fillna(0),axis=0)
# print(new_df.isna().sum())
ondates = pd.DataFrame({'date': pd.date_range(start=min(new_df.index), end=max(new_df.index), freq='D')})
# print(ondates.head())
ondates = pd.merge(ondates, new_df, how='left', on=['date'])
ondates.index = ondates['date']
del ondates['date']
ondates = ondates.apply(lambda x: x.fillna(0),axis=0)
print(ondates['2001-04-21':'2001-04-27'])

(12361, 191)
            stage_max_3019  stage_max_3021  stage_max_3024  stage_max_3027  \
date                                                                         
2001-04-21             0.0             0.0             0.0             0.0   
2001-04-22             0.0             0.0             0.0             0.0   
2001-04-23             0.0             0.0             0.0             0.0   
2001-04-24             0.0             0.0             0.0             0.0   
2001-04-25             0.0             0.0             0.0             0.0   
2001-04-26             0.0             0.0             0.0             0.0   
2001-04-27             0.0             0.0             0.0             0.0   

            stage_max_3028  stage_max_3029  stage_max_3030  stage_max_3031  \
date                                                                         
2001-04-21             0.0             0.0             0.0             0.0   
2001-04-22             0.0             0.0        

In [108]:
# load trained model for time saving
output_df = pd.DataFrame() # df for all gauges' predictions
# gauges loop
for lab in [f'stage_max_{i}' for i in [3019,3027,3028,3029,3030,3035,3041,3045,3050,3230,3036]]:
    print(lab)
    # shape new data
    val_frac = 0.01
    nt, nv, vt, vv, dm, dstd = make_dataset(ondates, val_frac, lab)
#     print(nt.shape, nv.shape, vt.shape, vv.shape)
    # load model
    multi_step_model = load_model(f"output/lena_" + lab + "_LSTM.h5")
#     print(multi_step_model.summary())
#     print('Multi-step model prediction shape:')
#     print(multi_step_model.predict(nt).shape)
    # predict
    pred = pd.DataFrame(multi_step_model.predict(nt) * dstd[df.columns.get_loc(lab)] + dm[df.columns.get_loc(lab)])
#     print(pred.head())
#     for i in range(future_target):
#         pred[i] = pred[i].shift(i)
#     pred.columns = [f'pred_{i}' for i in range(1, future_target + 1)]
    # get time index
    start_index = len(ondates) - (len(ondates) - int(len(ondates) * val_frac)) + past_history + future_target
    pred['date'] = ondates[start_index:].index.values
    pred['index'] = lab
    output_df = output_df.append(pred)
print(output_df.describe())

stage_max_3019
Train:  12656
Test:  127
(12656, 191)

 Input training data shape:
(60, 191) (12589, 7)

 Input test data shape:
(60, 191) (67, 7)
stage_max_3027
Train:  12656
Test:  127
(12656, 191)

 Input training data shape:
(60, 191) (12589, 7)

 Input test data shape:
(60, 191) (67, 7)
stage_max_3028
Train:  12656
Test:  127
(12656, 191)

 Input training data shape:
(60, 191) (12589, 7)

 Input test data shape:
(60, 191) (67, 7)
stage_max_3029
Train:  12656
Test:  127
(12656, 191)

 Input training data shape:
(60, 191) (12589, 7)

 Input test data shape:
(60, 191) (67, 7)
stage_max_3030
Train:  12656
Test:  127
(12656, 191)

 Input training data shape:
(60, 191) (12589, 7)

 Input test data shape:
(60, 191) (67, 7)
stage_max_3035
Train:  12656
Test:  127
(12656, 191)

 Input training data shape:
(60, 191) (12589, 7)

 Input test data shape:
(60, 191) (67, 7)
stage_max_3041
Train:  12656
Test:  127
(12656, 191)

 Input training data shape:
(60, 191) (12589, 7)

 Input test data sha

In [115]:
# solution_df = output_df
# solution_df['index'] = solution_df['index'].str[-4:].astype(int)
# solution_df.index = solution_df['date']
solution_df['2001-04-21':'2001-04-27']

,0,1,2,3,4,5,6,date,index,delta_stage_max_predicted
date,,,,,,,,,,
2001-04-21,19.252483,21.250595,25.599850,30.456059,23.722958,29.531677,39.953083,2001-04-21,3019,-6.889088
2001-04-22,17.077900,22.027718,27.979107,35.546143,33.087700,41.469681,51.245438,2001-04-22,3019,-2.174583
2001-04-23,12.169205,18.580044,24.966396,34.452766,41.632626,56.193382,67.527290,2001-04-23,3019,-4.908695
2001-04-24,15.393024,22.376823,38.096062,51.209251,52.823158,58.647346,62.596146,2001-04-24,3019,3.223820
2001-04-25,25.212601,30.500210,43.727501,56.257889,58.200306,59.515720,63.311241,2001-04-25,3019,9.819576
2001-04-26,22.341270,29.542351,44.922943,55.760895,53.463211,55.301655,58.003681,2001-04-26,3019,-2.871330
2001-04-27,21.440269,27.715168,39.241402,46.092697,43.626457,45.515343,47.270943,2001-04-27,3019,-0.901001
2001-04-21,56.325119,54.107475,58.923439,83.433838,107.355438,110.465424,123.632004,2001-04-21,3027,-8.549164
2001-04-22,78.176804,71.673523,74.452080,102.966324,128.201691,126.937134,138.157120,2001-04-22,3027,21.851685


In [113]:
# extract test periods' predictions
test_cp1 = pd.read_csv(r'data/lena/test_cp4.csv')
test_cp1['date'] = pd.to_datetime(test_cp1['date'])
print(test_cp1.head())

solution_df['delta_stage_max_predicted'] = solution_df[0].diff()
print(solution_df.head())
solution_cp1 = pd.merge(test_cp1, solution_df.loc[:,['index','delta_stage_max_predicted']], how = 'left', left_on=['date', 'station_id'], right_on=['date', 'index'])
print(solution_cp1.head(10))
solution_cp1.to_csv('results/submit_uh_cp4.csv')

   year  station_id  month  day       date  delta_stage_max
0  1993        3019      4  111 1993-04-21              NaN
1  1993        3019      4  112 1993-04-22              NaN
2  1993        3019      4  113 1993-04-23              NaN
3  1993        3019      4  114 1993-04-24              NaN
4  1993        3019      4  115 1993-04-25              NaN
                     0           1           2           3           4  \
date                                                                     
1985-07-14  143.359375  133.237305  123.466431  127.978790  127.485168   
1985-07-15  116.521439  113.026901  108.468292  111.108246  104.566208   
1985-07-16  115.331238  108.629547  101.700119  103.107986  102.461823   
1985-07-17  118.379692  110.961716  101.385437   97.934265  100.579788   
1985-07-18   92.310654  103.051819  103.691116   99.737518  104.867523   

                     5           6       date  index  \
date                                                   
1985-07-1

In [142]:
score_df = pd.DataFrame()
for lab in [f'stage_max_{i}' for i in [3019,3027,3028,3029,3030,3035,3041,3045,3050,3230,3036]]:
    print(lab)
    pred = pd.DataFrame(multi_step_model.predict(nt) * dstd[df.columns.get_loc(lab)] + dm[df.columns.get_loc(lab)])
    # 7-day forecast
    pred.columns = [f'pred_{i}' for i in range(1, future_target + 1)]
    start_index = len(ondates) - (len(ondates) - int(len(ondates) * val_frac)) + past_history + future_target
    pred.index = ondates[start_index:].index.values
#     print(pred.head())
    
#     get observations
    pred['fact'] = ondates[start_index:][lab]
    
#     plot    
#     fig, ax = plt.subplots()
#     subdf = pred['2013-04-01':'2013-06-30']
#     subdf.plot(ax=ax)
#     subdf['fact'].plot(ax=ax, lw=5)
#     subdf.diff().plot()
#     subdf['fact'].diff().plot(lw=5)
#     plt.show()
#     subdf.head(7)
    pred['index'] = lab[-4:]
    score_df = score_df.append(pred)
print(score_df.describe())

stage_max_3019
stage_max_3027
stage_max_3028
stage_max_3029
stage_max_3030
stage_max_3035
stage_max_3041
stage_max_3045
stage_max_3050
stage_max_3230
stage_max_3036
              pred_1         pred_2         pred_3         pred_4  \
count  138479.000000  138479.000000  138479.000000  138479.000000   
mean      220.445908     207.120560     212.829529     211.320938   
std       218.838150     214.994965     213.537537     208.903580   
min      -456.788513    -428.686554    -388.279144    -411.478882   
25%        91.615597      81.966560      86.991611      86.430698   
50%       187.976212     176.538895     182.462708     183.597870   
75%       297.934402     280.750183     288.993820     292.885529   
max      1437.879150    1500.158447    1470.960205    1487.799438   

              pred_5         pred_6         pred_7           fact  
count  138479.000000  138479.000000  138479.000000  138479.000000  
mean      217.567596     203.251175     191.233856     198.433343  
std      

In [147]:
score_df['fact_delta'] = score_df['fact'].diff(1)
score_df['pred_delta'] = score_df['pred_1'].diff(1)
score_df.to_csv('results/score_df.csv')